In [2]:
import numpy as np
import tensorflow as tf

In [27]:
model_name = 'temp_cnn_sampling_weighted'

### Load the data

In [49]:
NP_DATA_PATH = '/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/prepared_data/original_data.npz'

In [50]:
data = np.load(NP_DATA_PATH)
features = data['features']
field_id = data['field_id']
crop_id = data['crop_id']

features.shape, field_id.shape, crop_id.shape

((67557, 13, 14), (67557,), (67557,))

### Normalizing the data

In [51]:
mean_f = features.mean(axis=(0,1))
std_f = features.std(axis=(0,1))

mean_f, std_f

(array([0.04495004, 0.05522608, 0.08520251, 0.08335783, 0.13549377,
        0.25449966, 0.29252816, 0.29083016, 0.31650684, 0.31871104,
        0.26029542, 0.17459822, 0.5274778 , 3.82845028]),
 array([ 0.02151336,  0.023685  ,  0.02388389,  0.03328747,  0.03058358,
         0.0464853 ,  0.05491788,  0.05862865,  0.05730477,  0.05223269,
         0.05640826,  0.05882478,  0.18024394, 17.91834208]))

In [52]:
features = (features - mean_f)/std_f

## Spatial sampling

In [53]:
agg_methods = {
    'min': np.mean,
    'max': np.max,
    'mean': np.mean,
    'std': np.std
}

def sample_field(features, N=10, n_samples=1, methods=['mean']):
    agg_features_stack = []
    indices = np.arange(len(features))
    for _ in range(n_samples):
        sampled_i = np.random.choice(indices, size=N, replace=True)

        sampled_features = features[sampled_i]
        
        aggreagations = [agg_methods[method](sampled_features, axis=0) for method in methods]
        agg_features = np.concatenate(aggreagations, axis=-1)
        
        agg_features_stack.append(agg_features)
    
    return np.stack(agg_features_stack)

In [54]:
from tqdm.auto import tqdm

def sample_dataset(features, labels, field_ids, n=10, methods=['mean']):
    fields, counts = np.unique(field_ids, return_counts=True)
    
    features_sampled = []
    labels_sampled = []
    field_id_sampled = []
    weights_sampled = []
    for fid, c in zip(tqdm(fields), counts):
        n_samples = max(c//n, 3)

        field_mask = field_ids == fid
        field_features = features[field_mask]
        field_label = labels[field_mask][0]
        
        sampled_features = sample_field(field_features, N=n, n_samples=n_samples, methods=methods)
        sampled_labels = np.full(n_samples, field_label)
        sampled_fid = np.full(n_samples, fid)
        sampled_weights = np.full(n_samples, 1.0/n_samples)
        
        features_sampled.append(sampled_features)
        labels_sampled.append(sampled_labels)
        field_id_sampled.append(sampled_fid)
        weights_sampled.append(sampled_weights)
    
    features = np.concatenate(features_sampled, axis=0)
    labels = np.concatenate(labels_sampled, axis=0)
    field_ids = np.concatenate(field_id_sampled, axis=0)
    weights = np.concatenate(weights_sampled, axis=0)
    
    return features, labels, field_ids, weights

In [55]:
features_s, crop_id_s, field_id_s, weights_s = sample_dataset(features, crop_id, field_id, methods=['mean', 'min', 'max'])

In [56]:
features_s.shape

(14755, 13, 42)

In [57]:
# features_s = features_s[:, 5:11 ,:]

### Extract training data and split train, val

In [58]:
mask = crop_id_s != 0
features_valid = features_s[mask]
labels_valid = crop_id_s[mask] - 1
field_id_valid = field_id_s[mask]
weights_valid = weights_s[mask]

features_test = features_s[~mask]
field_id_test = field_id_s[~mask]

features_valid.shape, features_test.shape

((10363, 13, 42), (4392, 13, 42))

In [59]:
# # Compute field split and save
# val_ratio = 0.1
# random_state = np.random.RandomState(seed=42)

# fields = np.unique(field_id_valid)
# random_state.shuffle(fields)

# val_i = int(val_ratio * len(fields))
# fields_train = fields[val_i:]
# fields_val = fields[:val_i]
# fields_test = np.unique(field_id_test)

# # Save split
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/train.npy', fields_train)
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/val.npy', fields_val)
# np.save('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/test.npy', fields_test)
# fields_train.shape, fields_val.shape, fields_test.shape

In [60]:
# Load training field IDs (split)
fields_train = np.load('/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/split/train.npy')

In [61]:
train_mask = np.isin(field_id_valid, fields_train)
print(f'Ratio of pixels in train: {np.mean(train_mask)}')

features_train = features_valid[train_mask]
labels_train = labels_valid[train_mask]
field_id_train = field_id_valid[train_mask]
weights_train = weights_valid[train_mask]

features_val = features_valid[~train_mask]
labels_val = labels_valid[~train_mask]
field_id_val = field_id_valid[~train_mask]
weights_val = weights_valid[~train_mask]

features_train.shape, features_val.shape

Ratio of pixels in train: 0.8999324519926662


((9326, 13, 42), (1037, 13, 42))

In [62]:
# Flattened features
features_train_flat = features_train.reshape((features_train.shape[0], -1))
features_val_flat = features_val.reshape((features_val.shape[0], -1))

### Train model

In [63]:
from eoflow.models import TempCNNModel, BiRNN, TransformerEncoder

In [64]:
# TempCNN model

model_config = {
    'learning_rate': 0.1,
    'n_classes': 7,
    'keep_prob': 0.5,
    'nb_conv_stacks': 2
}
model = TempCNNModel(model_config)

In [65]:
# BiRNN model

# model_config = {
#     'learning_rate': 0.1,
#     'n_classes': 7,
#     'rnn_layer': 'lstm',
#     'keep_prob': 0.5
# }
# model = BiRNN(model_config)

In [66]:
# Dense Model

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(7, activation='softmax')
# ])

In [67]:
# Class weights
# classes = np.unique(labels_valid, return_counts=True)
# class_probs = classes[1]/np.sum(classes[1])
# class_inv_probs = 1.0 / class_probs
# weights = {i:inv_prob for i, inv_prob in enumerate(class_inv_probs)}
# weights

In [68]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [69]:
# Temporal
model.fit(
    x=features_train, 
    y=labels_train, 
    validation_data=(features_val, labels_val),
    batch_size=256,
    epochs=300,
    sample_weight=weights_train
)

# Non-temporal
# model.fit(
#     x=features_train_flat, 
#     y=labels_train, 
#     validation_data=(features_val_flat, labels_val),
#     batch_size=1024,
#     epochs=10)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 13, 42)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 13, 16)            3376      
_________________________________________________________________
activation_6 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 13, 16)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 13, 16)            1296      
_________________________________________________________________
activation_7 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 16)            0   

2020-03-10 16:51:24,083 WARNING sample_weight modes were coerced from
  ...
    to  
  ['...']


Train on 9326 samples, validate on 1037 samples
Epoch 1/300
9326/9326 [==============================] - 2s 196us/sample - loss: 0.7126 - accuracy: 0.3507 - val_loss: 1.5944 - val_accuracy: 0.4937
Epoch 2/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.5177 - accuracy: 0.4165 - val_loss: 1.5503 - val_accuracy: 0.4976
Epoch 3/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.4853 - accuracy: 0.4519 - val_loss: 1.5015 - val_accuracy: 0.4937
Epoch 4/300
9326/9326 [==============================] - 1s 78us/sample - loss: 0.4749 - accuracy: 0.4724 - val_loss: 1.4807 - val_accuracy: 0.5130
Epoch 5/300
9326/9326 [==============================] - 1s 76us/sample - loss: 0.4636 - accuracy: 0.4916 - val_loss: 1.4570 - val_accuracy: 0.5140
Epoch 6/300
9326/9326 [==============================] - 1s 73us/sample - loss: 0.4557 - accuracy: 0.4990 - val_loss: 1.4366 - val_accuracy: 0.5477
Epoch 7/300
9326/9326 [==============================] - 1s 74u

Epoch 56/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3838 - accuracy: 0.5765 - val_loss: 1.2015 - val_accuracy: 0.5747
Epoch 57/300
9326/9326 [==============================] - 1s 70us/sample - loss: 0.3834 - accuracy: 0.5798 - val_loss: 1.2040 - val_accuracy: 0.5767
Epoch 58/300
9326/9326 [==============================] - 1s 76us/sample - loss: 0.3825 - accuracy: 0.5786 - val_loss: 1.2006 - val_accuracy: 0.5815
Epoch 59/300
9326/9326 [==============================] - 1s 78us/sample - loss: 0.3803 - accuracy: 0.5822 - val_loss: 1.1858 - val_accuracy: 0.5902
Epoch 60/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3816 - accuracy: 0.5804 - val_loss: 1.1947 - val_accuracy: 0.5853
Epoch 61/300
9326/9326 [==============================] - 1s 71us/sample - loss: 0.3810 - accuracy: 0.5797 - val_loss: 1.1824 - val_accuracy: 0.5863
Epoch 62/300
9326/9326 [==============================] - 1s 71us/sample - loss: 0.3810 - accuracy: 0.5826

Epoch 165/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3497 - accuracy: 0.6072 - val_loss: 1.1731 - val_accuracy: 0.6095
Epoch 166/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3524 - accuracy: 0.6024 - val_loss: 1.1662 - val_accuracy: 0.5998
Epoch 167/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3513 - accuracy: 0.5974 - val_loss: 1.1733 - val_accuracy: 0.6017
Epoch 168/300
9326/9326 [==============================] - ETA: 0s - loss: 0.3471 - accuracy: 0.60 - 1s 72us/sample - loss: 0.3500 - accuracy: 0.6025 - val_loss: 1.1669 - val_accuracy: 0.5988
Epoch 169/300
9326/9326 [==============================] - 1s 71us/sample - loss: 0.3496 - accuracy: 0.6059 - val_loss: 1.1579 - val_accuracy: 0.5979
Epoch 170/300
9326/9326 [==============================] - 1s 71us/sample - loss: 0.3494 - accuracy: 0.6009 - val_loss: 1.1529 - val_accuracy: 0.5969
Epoch 171/300
9326/9326 [==============================] -

Epoch 274/300
9326/9326 [==============================] - 1s 78us/sample - loss: 0.3374 - accuracy: 0.6189 - val_loss: 1.1560 - val_accuracy: 0.5998
Epoch 275/300
9326/9326 [==============================] - 1s 75us/sample - loss: 0.3384 - accuracy: 0.6213 - val_loss: 1.1569 - val_accuracy: 0.6085
Epoch 276/300
9326/9326 [==============================] - 1s 73us/sample - loss: 0.3353 - accuracy: 0.6183 - val_loss: 1.1641 - val_accuracy: 0.6114
Epoch 277/300
9326/9326 [==============================] - 1s 77us/sample - loss: 0.3388 - accuracy: 0.6117 - val_loss: 1.1673 - val_accuracy: 0.6085
Epoch 278/300
9326/9326 [==============================] - 1s 73us/sample - loss: 0.3365 - accuracy: 0.6172 - val_loss: 1.1600 - val_accuracy: 0.6114
Epoch 279/300
9326/9326 [==============================] - 1s 72us/sample - loss: 0.3342 - accuracy: 0.6171 - val_loss: 1.1700 - val_accuracy: 0.6027
Epoch 280/300
9326/9326 [==============================] - 1s 73us/sample - loss: 0.3369 - accuracy:

### Random forest

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

random_state = 7
rf = RandomForestClassifier(n_estimators=500, random_state=random_state, n_jobs=-1)

rf.fit(features_train_flat, labels_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=7, verbose=0,
                       warm_start=False)

### Predictions

In [35]:
preds_rf = rf.predict_proba(features_val_flat)
preds_dnn = model.predict(features_val)

In [36]:
preds_rf.shape, preds_dnn.shape

((1037, 7), (1037, 7))

In [37]:
def one_hot(labels, num_classes):
    """ One-hot encodes integer labels for a given number of classes. """
    
    one_hot_labels = np.zeros((labels.size, num_classes))
    one_hot_labels[np.arange(labels.size),labels] = 1
    
    return one_hot_labels

In [38]:
import pandas as pd
def field_predictions(predictions, labels, field_ids):
    """ Groups pixel-wise predictions into field predictions by averaging. """
    
    df = pd.DataFrame(predictions)
    df['CROP_ID'] = labels
    df['FIELD_ID'] = field_ids
    
    df_grouped = df.groupby('FIELD_ID').mean()
    field_preds = df_grouped.drop('CROP_ID', axis=1).values
    field_labels = df_grouped['CROP_ID'].values
    
    return field_preds, field_labels, df_grouped.index

In [39]:
# Create onehot labels for val
one_hot_labels_val = one_hot(labels_val, 7)

### Pixel log loss

In [40]:
rf_loss = log_loss(one_hot_labels_val, preds_rf)
dnn_loss = log_loss(one_hot_labels_val, preds_dnn)

print(f'Random forest: {rf_loss}')
print(f'DNN: {dnn_loss}')

Random forest: 1.2297453785931451
DNN: 1.1278737355345252


### Field log loss

In [41]:
preds_rf_field, lbl_rf_field, field_id_rf = field_predictions(preds_rf, labels_val, field_id_val)
preds_dnn_field, lbl_dnn_field, field_id_dnn = field_predictions(preds_dnn, labels_val, field_id_val)

In [42]:
lbl_rf_field_oh = one_hot(lbl_rf_field, 7)
lbl_dnn_field_oh = one_hot(lbl_dnn_field, 7)

In [43]:
rf_loss_field = log_loss(lbl_rf_field_oh, preds_rf_field)
dnn_loss_field = log_loss(lbl_dnn_field_oh, preds_dnn_field)

print(f'Random forest: {rf_loss_field}')
print(f'DNN: {dnn_loss_field}')

Random forest: 1.2418810512159248
DNN: 1.1261859473391869


## Prepare predictions for analysis

In [44]:
import pandas as pd

In [45]:
def prepare_predictions(preds, labels, field_ids):
    preds_l = np.argmax(preds, axis=1)

    df = pd.DataFrame(preds, columns=[f'Crop_ID_{i+1}' for i in range(7)])
    df['Field_ID'] = field_ids
    df['prediction'] = preds_l+1
    df['label'] = labels+1
    
    df = df.set_index('Field_ID')
    
    return df

In [46]:
df_rf = prepare_predictions(preds_rf_field, lbl_rf_field, field_id_rf)
df_dnn = prepare_predictions(preds_dnn_field, lbl_dnn_field, field_id_dnn)
df_rf

,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,prediction,label
Field_ID,,,,,,,,,
1,0.342000,0.446000,0.010000,0.086000,0.060000,0.038000,0.018000,2,1
27,0.539333,0.296667,0.025333,0.037333,0.024000,0.073333,0.004000,1,2
33,0.301333,0.310667,0.108000,0.118667,0.061333,0.031333,0.068667,2,2
44,0.336667,0.510667,0.037333,0.050667,0.017333,0.036667,0.010667,2,1
50,0.270000,0.478667,0.014000,0.127333,0.051333,0.045333,0.013333,2,5
...,...,...,...,...,...,...,...,...,...
4717,0.538000,0.300000,0.026667,0.040667,0.069333,0.019333,0.006000,1,1
4733,0.538800,0.074800,0.046600,0.191800,0.039000,0.090200,0.018800,1,1
4735,0.344000,0.187333,0.036000,0.188667,0.072000,0.164000,0.008000,1,4


In [47]:
np.mean(df_dnn['prediction'] == df_dnn['label'])

0.6097560975609756

In [71]:
df_dnn.to_csv(f'/mnt/azrael/spatialdata/projects/eo_data/iclr-2020-challange/mlhub/val_predictions/{model_name}.csv')

## Prepare results for submission

In [48]:
# Train best model on whole training dataset (without val)
# TempCNN model

model_config = {
    'learning_rate': 0.1,
    'n_classes': 7,
    'keep_prob': 0.5,
    'nb_conv_stacks': 2
}
model_final = TempCNNModel(model_config)
model_final.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [49]:
model_final.fit(
    x=features_valid, 
    y=labels_valid,
    batch_size=256,
    epochs=300,
    sample_weight=weights_valid,
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 13, 42)]          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 16)            3376      
_________________________________________________________________
activation_3 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 13, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 13, 16)            1296      
_________________________________________________________________
activation_4 (Activation)    (None, 13, 16)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 13, 16)            0   

2020-03-10 16:24:23,359 WARNING sample_weight modes were coerced from
  ...
    to  
  ['...']


Train on 10363 samples
Epoch 1/300
10363/10363 [==============================] - 2s 172us/sample - loss: 0.7345 - accuracy: 0.3323
Epoch 2/300
10363/10363 [==============================] - 1s 75us/sample - loss: 0.5092 - accuracy: 0.4178
Epoch 3/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.4854 - accuracy: 0.4445
Epoch 4/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.4735 - accuracy: 0.4685
Epoch 5/300
10363/10363 [==============================] - 1s 68us/sample - loss: 0.4640 - accuracy: 0.4883
Epoch 6/300
10363/10363 [==============================] - 1s 75us/sample - loss: 0.4555 - accuracy: 0.5014
Epoch 7/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.4485 - accuracy: 0.5147
Epoch 8/300
10363/10363 [==============================] - 1s 68us/sample - loss: 0.4436 - accuracy: 0.5260
Epoch 9/300
10363/10363 [==============================] - 1s 68us/sample - loss: 0.4393 - accuracy: 0.5227
Epoc

10363/10363 [==============================] - 1s 69us/sample - loss: 0.3748 - accuracy: 0.5860
Epoch 77/300
10363/10363 [==============================] - 1s 74us/sample - loss: 0.3744 - accuracy: 0.5889
Epoch 78/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.3767 - accuracy: 0.5850
Epoch 79/300
10363/10363 [==============================] - 1s 71us/sample - loss: 0.3744 - accuracy: 0.5859
Epoch 80/300
10363/10363 [==============================] - 1s 70us/sample - loss: 0.3739 - accuracy: 0.5835
Epoch 81/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.3761 - accuracy: 0.5828
Epoch 82/300
10363/10363 [==============================] - 1s 71us/sample - loss: 0.3742 - accuracy: 0.5864
Epoch 83/300
10363/10363 [==============================] - 1s 73us/sample - loss: 0.3772 - accuracy: 0.5826
Epoch 84/300
10363/10363 [==============================] - 1s 68us/sample - loss: 0.3727 - accuracy: 0.5848
Epoch 85/300
10363/10363 [======

10363/10363 [==============================] - 1s 70us/sample - loss: 0.3423 - accuracy: 0.6120
Epoch 225/300
10363/10363 [==============================] - 1s 70us/sample - loss: 0.3465 - accuracy: 0.6156
Epoch 226/300
10363/10363 [==============================] - 1s 73us/sample - loss: 0.3447 - accuracy: 0.6134
Epoch 227/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.3436 - accuracy: 0.6115
Epoch 228/300
10363/10363 [==============================] - 1s 70us/sample - loss: 0.3429 - accuracy: 0.6121
Epoch 229/300
10363/10363 [==============================] - 1s 72us/sample - loss: 0.3457 - accuracy: 0.6109
Epoch 230/300
10363/10363 [==============================] - 1s 69us/sample - loss: 0.3445 - accuracy: 0.6135
Epoch 231/300
10363/10363 [==============================] - 1s 67us/sample - loss: 0.3411 - accuracy: 0.6144
Epoch 232/300
10363/10363 [==============================] - 1s 74us/sample - loss: 0.3444 - accuracy: 0.6124
Epoch 233/300
10363/1036

Test on val

In [55]:
preds_val = model_final.predict(features_val)

In [56]:
one_hot_labels_val = one_hot(labels_val, 7)
log_loss(one_hot_labels_val, preds_val)

0.8696593886087294

Predict on test data

In [60]:
preds_final = model_final.predict(features_test)

In [61]:
# Groups pixel-wise predictions into field predictions by averaging.
    
df_pred = pd.DataFrame(preds_final, columns=[f'Crop_ID_{i+1}' for i in range(7)])
df_pred['Field_ID'] = field_id_test
    
df_grouped = df_pred.groupby('Field_ID').mean()
df_grouped

,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7
Field_ID,,,,,,,
3,0.093878,0.741820,0.018600,0.058965,0.010796,0.007072,0.068869
6,0.092141,0.853336,0.000131,0.013566,0.013991,0.025687,0.001148
11,0.087642,0.674603,0.010626,0.132801,0.056422,0.003938,0.033968
13,0.243931,0.547923,0.016267,0.029965,0.114173,0.034280,0.013460
14,0.048479,0.865151,0.042108,0.007048,0.008549,0.000287,0.028379
...,...,...,...,...,...,...,...
4785,0.909692,0.070041,0.006883,0.007005,0.002077,0.002031,0.002272
4788,0.922520,0.023131,0.007554,0.028200,0.004992,0.013580,0.000022
4790,0.790623,0.102938,0.008065,0.026936,0.042148,0.028407,0.000883


In [62]:
df_grouped.to_csv('submissions/temp_cnn_sampling_weighted.csv')

In [63]:
model_final.save_weights('models/temp_cnn_sampling_weighted.hdf5')